In [1]:
import os
import numpy as np
import pandas as pd
from implicit.als import AlternatingLeastSquares

## 1) 데이터 준비와 전처리

Movielens 데이터는 `rating.dat` 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head(5)

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## 2) 분석해 봅시다.

In [7]:
#ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [8]:
# ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [9]:
#가장 인기 있는 영화 5개(인기순)
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(5)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
Name: user_id, dtype: int64

In [10]:
# 유저별 몇 개의 영화 평점을 남겼는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [11]:
# 유저별 별점 중앙값에 대한 통계
user_median = ratings.groupby('user_id')['counts'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: counts, dtype: float64

## 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [12]:
# 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰야합니다.
my_favorite = ['Matrix, The (1999)','Pulp Fiction (1994)','Fight Club (1999)',
               'Godfather, The (1972)','Godfather: Part II, The (1974)']

# 'zimin'이라는 user_id가 위 영화들의 평점을 모두 5점을 주었다고 가정하겠습니다.
my_recommend_list = pd.DataFrame({'user_id': ['zimin']*5, 
                                  'movie_id': [int(movies.loc[movies['title'] == f].movie_id.values) 
                                               for f in my_favorite],
                                  'counts':[5]*5})

if not ratings.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings = ratings.append(my_recommend_list)               # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)    # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts,timestamp
1000203,6040,1090,3,956715518.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
0,zimin,2571,5,NaN
1,zimin,296,5,NaN
2,zimin,2959,5,NaN
3,zimin,858,5,NaN
4,zimin,1221,5,NaN


### 모델에 활용하기 위한 전처리

In [13]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [14]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):  # 모든 row가 정상적으로 인덱싱되었다면
    print("user id col indexing OK!!")
    ratings['user_id'] = temp_user_data  # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print("user_id col indexing FAIL!!")

# movie_to_idx을 통해 movie 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print("title col indexing OK!!")
    ratings['movie_id'] = temp_movie_data
else:
    print("title col indexing FAIL!!")

#timestamp열은 사용하지 않아 제거
ratings = ratings.drop(['timestamp'],axis=1)

ratings

user id col indexing OK!!
title col indexing OK!!


,user_id,movie_id,counts
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
0,6039,124,5
1,6039,222,5
2,6039,273,5
3,6039,607,5


## 4) CSR matrix를 직접 만들어 봅시다.

In [15]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['counts'], (ratings.user_id,ratings.movie_id)),shape = (num_user,num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [16]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [17]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [18]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [19]:
movie_to_title = {k+1:v for k,v in enumerate(movies['title'])}
title_to_movie = {v:k for k,v in movie_to_title.items()}

In [20]:
zimin, jumanji = user_to_idx['zimin'], movie_to_idx[title_to_movie['Jumanji (1995)']]

zimin_vector, jumanji_vector = als_model.user_factors[zimin], als_model.item_factors[jumanji]

In [21]:
zimin_vector

array([ 0.13136217, -0.27002016, -0.36802787, -0.5497031 ,  0.5850868 ,
       -0.12906714,  0.46075583, -0.05837251,  0.0355358 , -0.00224197,
        0.7368139 ,  0.30757517, -0.3306971 , -0.10928951,  0.7101775 ,
       -0.3197877 ,  0.5992106 ,  0.680883  ,  0.09447455,  0.33498135,
        0.15681492,  0.09142683,  0.40982798,  0.13147196, -0.00668541,
       -0.8258568 , -0.3642349 , -0.08945084, -0.44484687,  0.9083071 ,
       -0.31315887,  0.1700081 ,  0.06051848,  0.4077211 , -0.67717284,
        0.31498623, -0.25110772, -0.17283814,  0.13910209, -0.00234214,
        0.02605884,  0.278393  , -0.32154554,  0.19619423, -0.4466014 ,
       -1.1473708 , -0.06068118,  0.58960724, -0.3378796 , -0.15304056,
       -0.1767513 ,  0.792226  , -0.6871965 , -0.9373582 ,  0.348522  ,
        0.58170396, -0.60302705, -0.42372087,  0.15026695,  0.64247006,
       -1.3393962 ,  0.5949769 ,  0.2678427 ,  0.12202346,  0.2889085 ,
       -0.70108426,  0.50521386,  1.0284338 ,  0.1335422 ,  0.09

In [22]:
jumanji_vector

array([-0.00744214,  0.00121943, -0.00181348, -0.00459873,  0.00042809,
        0.0073741 ,  0.00393523,  0.01332891,  0.03810703,  0.0167053 ,
        0.00068453,  0.01541445,  0.03921291,  0.00603828,  0.00178577,
        0.02392749,  0.02308008,  0.01063215,  0.00367634, -0.0003247 ,
        0.02912643,  0.00335166,  0.0159455 ,  0.00742424, -0.00145958,
       -0.00400946,  0.00884386, -0.00369312, -0.00376901,  0.0190772 ,
        0.02745699,  0.01917765, -0.00145298, -0.02430329,  0.0063698 ,
       -0.00029239,  0.00775581,  0.01694181,  0.01051497, -0.00563265,
        0.02402639,  0.02145174, -0.00678872,  0.03118223,  0.01617274,
        0.00322242,  0.01642855,  0.02482675,  0.01314717,  0.0079575 ,
        0.01210053, -0.01050726, -0.001684  ,  0.02135758,  0.01002378,
       -0.01780572, -0.00730166, -0.00391872, -0.00244144, -0.00361992,
        0.02660478,  0.00098311,  0.00346177,  0.01480605, -0.02169392,
        0.02598581,  0.0011377 ,  0.01684798,  0.02678942, -0.00

In [23]:
# zimin과 jumanji_vector를 내적하는 코드
np.dot(zimin_vector, jumanji_vector)

0.050803043

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [24]:
favorite_movie = title_to_movie["Back to the Future (1985)"]
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(1428, 0.99999994),
 (784, 0.7421326),
 (1836, 0.72292125),
 (1440, 0.71695),
 (1832, 0.70692873),
 (2279, 0.70325035),
 (2506, 0.70068985),
 (1833, 0.6916113),
 (1292, 0.6814557),
 (1356, 0.6738838),
 (2271, 0.6602269),
 (1472, 0.64912224),
 (1834, 0.64845175),
 (1848, 0.64751095),
 (2840, 0.63938123)]

In [25]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[movie_to_title[i[0]] for i in similar_movie]

['SubUrbia (1997)',
 "My Life and Times With Antonin Artaud (En compagnie d'Antonin Artaud) (1993)",
 'Henry Fool (1997)',
 'Donnie Brasco (1997)',
 'Children of Heaven, The (Bacheha-Ye Aseman) (1997)',
 'Pope of Greenwich Village, The (1984)',
 'Out-of-Towners, The (1999)',
 'Dear Jesse (1997)',
 'Santa with Muscles (1996)',
 'Star Trek IV: The Voyage Home (1986)',
 "I'll Be Home For Christmas (1998)",
 '8 Heads in a Duffel Bag (1997)',
 'Dream for an Insomniac (1996)',
 'Buffalo 66 (1998)',
 "Boys Don't Cry (1999)"]

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [26]:
title_recommended = als_model.recommend(user_to_idx['zimin'], csr_data, N=20, filter_already_liked_items=True)

In [27]:
[movie_to_title[i[0]] for i in title_recommended]

['Losing Isaiah (1995)',
 'Boomerang (1992)',
 'It Takes Two (1995)',
 'Guardian Angel (1994)',
 'Ed Wood (1994)',
 'Casper (1995)',
 'Cowboy Way, The (1994)',
 'Pocahontas (1995)',
 'Vampire in Brooklyn (1995)',
 'Shallow Grave (1994)',
 'Poison Ivy II (1995)',
 'Safe Passage (1994)',
 'Boys of St. Vincent, The (1993)',
 'City Hall (1996)',
 'Circle of Friends (1995)',
 'If Lucy Fell (1996)',
 'Dream Man (1995)',
 'Three Wishes (1995)',
 'Two Bits (1995)',
 'Restoration (1995)']

---
## 회고 
ㅁ